In [2]:
import os
import django

# The following lines are here to avoid `SynchronousOnlyOperation` exception
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'scraper.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

from articles.models import Article

In [3]:
from spacy.lang.ky import Kyrgyz

spacy_nlp = Kyrgyz()
spacy_nlp.add_pipe("sentencizer")

In [5]:
arts = Article.objects.all()
print(len(arts))

16826


In [15]:
def append_sentences_to_file(sentences):
    with open('all_texts.txt', 'a') as file:
        for sentence in sentences:
            text = sentence.text.strip()
            file.write(text + '\n')

In [16]:
for art in arts:
    doc = spacy_nlp(art.text)
    append_sentences_to_file(doc.sents)